In [1]:
# import necessary packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# google cloud packages
from google.cloud import storage
from google.colab import auth
from google.cloud import bigquery
from google.colab import drive

# install google cloud storage library
!pip install google-cloud-storage

# authenticate with google cloud storage
auth.authenticate_user()

In [2]:
# setup code for use with google cloud storage and bigquery

# google cloud project setup
project_id = "mimic-project-451516"

# instantiate client for google cloud storage
gcs_client = storage.Client(project=project_id)

# instantiate client for bigquery
bq_client = bigquery.Client(project=project_id)

# instantiate MIMIC-III storage bucket
bucket_name = "mimiciii-1.4.physionet.org"

In [3]:
# check available files in the bucket by printing list of files
print(f"Files in GCS bucket: {bucket_name}")

# instantiate bucket and store files in blob
gcs_bucket = gcs_client.bucket(bucket_name, user_project=project_id)
blobs = gcs_bucket.list_blobs()
file_list = [blob.name for blob in blobs]

# format so that files and their list index are present for quick access later
df_files = pd.DataFrame({
    "Index": range(len(file_list)),
    "Filename": file_list
})

# filter only on csv files that contain patient data
df_files = df_files[df_files['Filename'].str.endswith('.csv.gz')]

print(df_files.to_string(index=False))

Files in GCS bucket: mimiciii-1.4.physionet.org
 Index                  Filename
     0         ADMISSIONS.csv.gz
     1            CALLOUT.csv.gz
     2         CAREGIVERS.csv.gz
     3        CHARTEVENTS.csv.gz
     4          CPTEVENTS.csv.gz
     5     DATETIMEEVENTS.csv.gz
     6      DIAGNOSES_ICD.csv.gz
     7           DRGCODES.csv.gz
     8              D_CPT.csv.gz
     9    D_ICD_DIAGNOSES.csv.gz
    10   D_ICD_PROCEDURES.csv.gz
    11            D_ITEMS.csv.gz
    12         D_LABITEMS.csv.gz
    13           ICUSTAYS.csv.gz
    14     INPUTEVENTS_CV.csv.gz
    15     INPUTEVENTS_MV.csv.gz
    16          LABEVENTS.csv.gz
    18 MICROBIOLOGYEVENTS.csv.gz
    19         NOTEEVENTS.csv.gz
    20       OUTPUTEVENTS.csv.gz
    21           PATIENTS.csv.gz
    22      PRESCRIPTIONS.csv.gz
    23 PROCEDUREEVENTS_MV.csv.gz
    24     PROCEDURES_ICD.csv.gz
    26           SERVICES.csv.gz
    28          TRANSFERS.csv.gz


In [4]:
# define function to load each csv file either by downloading or using Big Query query
def load_file_results_to_df(file_path=None, query=None, bucket=gcs_bucket, client=None):

    # load csv file directly from google cloud storage (for files that are smaller)
    if file_path:
        blob = bucket.blob(file_path)
        blob.download_to_filename(file_path)
        print(f"Download complete: {file_path}")
        df = pd.read_csv(file_path, compression="gzip")

    # load csv file after applying query using bigquery (for larger files)
    elif query:
        df = bq_client.query(query).to_dataframe()
        print("Query executed and loaded to DataFrame using BigQuery")

    # count number of rows in dataframe and print message
    row_count = len(df)
    print(f"DataFrame with {row_count} row(s) has been returned.")

    return df

ADMINISTRATIVE DATASETS

CALLOUT DATASET ANALYSIS

In [5]:
# SQL query to select callouts dataset

callout_query = f"""
SELECT *
FROM `physionet-data.mimiciii_clinical.callout`
"""

In [6]:
# load callout dataframe from bigquery
callout_df = load_file_results_to_df(query=callout_query, bucket=gcs_bucket, client=bq_client)

Query executed and loaded to DataFrame using BigQuery
DataFrame with 34499 row(s) has been returned.


In [7]:
# describe callout dataframe and print some rows of dataframe
callout_df.info()
callout_df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34499 entries, 0 to 34498
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   ROW_ID                  34499 non-null  Int64         
 1   SUBJECT_ID              34499 non-null  Int64         
 2   HADM_ID                 34499 non-null  Int64         
 3   SUBMIT_WARDID           34498 non-null  Int64         
 4   SUBMIT_CAREUNIT         5122 non-null   object        
 5   CURR_WARDID             34498 non-null  Int64         
 6   CURR_CAREUNIT           34498 non-null  object        
 7   CALLOUT_WARDID          34499 non-null  Int64         
 8   CALLOUT_SERVICE         34499 non-null  object        
 9   REQUEST_TELE            34499 non-null  Int64         
 10  REQUEST_RESP            34499 non-null  Int64         
 11  REQUEST_CDIFF           34499 non-null  Int64         
 12  REQUEST_MRSA            34499 non-null  Int64 

,ROW_ID,SUBJECT_ID,HADM_ID,SUBMIT_WARDID,SUBMIT_CAREUNIT,CURR_WARDID,CURR_CAREUNIT,CALLOUT_WARDID,CALLOUT_SERVICE,REQUEST_TELE,...,CALLOUT_STATUS,CALLOUT_OUTCOME,DISCHARGE_WARDID,ACKNOWLEDGE_STATUS,CREATETIME,UPDATETIME,ACKNOWLEDGETIME,OUTCOMETIME,FIRSTRESERVATIONTIME,CURRENTRESERVATIONTIME
0,24702,68065,143576,14,TSICU,14,TSICU,24,GU,0,...,Inactive,Cancelled,<NA>,Acknowledged,2197-03-06 13:49:38,2197-03-06 14:29:03,2197-03-06 13:50:55,2197-03-06 14:29:03,NaT,NaT
1,1329,3445,196939,14,None,24,TSICU,24,GU,0,...,Inactive,Discharged,24,Acknowledged,2141-07-05 09:07:25,2141-07-05 09:07:25,2141-07-05 09:44:05,2141-07-05 15:10:02,NaT,NaT
2,1005,2558,163053,14,None,24,TSICU,24,GU,0,...,Inactive,Discharged,24,Acknowledged,2187-10-05 08:43:03,2187-10-05 08:49:05,2187-10-05 09:14:42,2187-10-05 22:10:01,NaT,NaT
3,1010,2574,179700,14,None,24,TSICU,1,GU,0,...,Inactive,Discharged,24,Revised,2133-11-27 10:20:13,2133-11-28 17:20:58,NaT,2133-11-28 17:25:02,NaT,NaT
4,5211,13347,171387,14,None,24,TSICU,24,GU,0,...,Inactive,Discharged,24,Acknowledged,2134-12-30 07:51:05,2134-12-30 09:26:49,2134-12-30 09:59:30,2134-12-30 15:40:02,NaT,NaT
5,4487,11505,162933,14,None,24,TSICU,24,GU,0,...,Inactive,Discharged,24,Acknowledged,2175-02-14 08:25:58,2175-02-14 08:25:58,2175-02-14 08:31:04,2175-02-14 18:10:04,NaT,NaT
6,16136,40934,105418,14,None,24,TSICU,1,GU,0,...,Inactive,Discharged,24,Acknowledged,2114-10-05 12:35:13,2114-10-05 12:35:13,2114-10-05 12:37:40,2114-10-05 18:44:13,2114-10-05 15:31:41,NaT
7,22350,60355,154806,14,None,24,TSICU,24,GU,0,...,Inactive,Discharged,24,Acknowledged,2106-10-10 10:42:16,2106-10-10 11:34:59,2106-10-10 11:38:01,2106-10-10 18:40:25,2106-10-10 15:10:25,NaT
8,19568,51796,133514,14,None,24,TSICU,24,GU,0,...,Inactive,Discharged,24,Acknowledged,2146-10-30 09:06:56,2146-10-30 09:06:56,2146-10-30 09:07:35,2146-10-30 19:25:29,2146-10-30 17:10:26,NaT
9,18391,48115,138322,14,None,24,TSICU,1,GU,0,...,Inactive,Discharged,24,Acknowledged,2126-07-18 12:39:31,2126-07-18 12:39:31,2126-07-18 12:47:32,2126-07-18 17:25:04,2126-07-18 13:55:05,NaT


In [8]:
# check for duplicate patients
duplicate_subject_id_count = callout_df.duplicated(subset=['SUBJECT_ID']).sum()
print(f"Number of duplicate SUBJECT_IDs is: {duplicate_subject_id_count}")

# check for duplicate admissions
duplicate_hadm_id_count = callout_df.duplicated(subset=['HADM_ID']).sum()
print(f"Number of duplicate HADM_IDs is: {duplicate_hadm_id_count}")

# reviewing missingness across the admissions dataset
callout_null_values_df = callout_df.isnull().sum().reset_index()
callout_null_values_df.columns = ['Column', 'Missing Count']
callout_null_values_df.sort_values(by='Missing Count', ascending=False)

Number of duplicate SUBJECT_IDs is: 11628
Number of duplicate HADM_IDs is: 5767


,Column,Missing Count
23,CURRENTRESERVATIONTIME,33335
4,SUBMIT_CAREUNIT,29377
22,FIRSTRESERVATIONTIME,19239
16,DISCHARGE_WARDID,4532
20,ACKNOWLEDGETIME,1792
3,SUBMIT_WARDID,1
5,CURR_WARDID,1
6,CURR_CAREUNIT,1
7,CALLOUT_WARDID,0
0,ROW_ID,0


In [9]:
print(callout_df['REQUEST_TELE'].value_counts())
print(callout_df['REQUEST_RESP'].value_counts())
print(callout_df['REQUEST_CDIFF'].value_counts())
print(callout_df['REQUEST_MRSA'].value_counts())
print(callout_df['REQUEST_VRE'].value_counts())

REQUEST_TELE
1    19220
0    15279
Name: count, dtype: Int64
REQUEST_RESP
0    34208
1      291
Name: count, dtype: Int64
REQUEST_CDIFF
0    33485
1     1014
Name: count, dtype: Int64
REQUEST_MRSA
0    31163
1     3336
Name: count, dtype: Int64
REQUEST_VRE
0    33104
1     1395
Name: count, dtype: Int64


ICUSTAYS DATASET ANALYSIS

In [10]:
# SQL query to select all icustays records
icustays_query = f"""
SELECT *
FROM `physionet-data.mimiciii_clinical.icustays`
"""

In [11]:
# load icustays dataframe to memory
icustays_df = load_file_results_to_df(query=icustays_query, bucket=gcs_bucket, client=bq_client)

Query executed and loaded to DataFrame using BigQuery
DataFrame with 61532 row(s) has been returned.


In [12]:
# desscribe icustays dataframe and print first rows
icustays_df.info()
icustays_df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61532 entries, 0 to 61531
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   ROW_ID          61532 non-null  Int64         
 1   SUBJECT_ID      61532 non-null  Int64         
 2   HADM_ID         61532 non-null  Int64         
 3   ICUSTAY_ID      61532 non-null  Int64         
 4   DBSOURCE        61532 non-null  object        
 5   FIRST_CAREUNIT  61532 non-null  object        
 6   LAST_CAREUNIT   61532 non-null  object        
 7   FIRST_WARDID    61532 non-null  Int64         
 8   LAST_WARDID     61532 non-null  Int64         
 9   INTIME          61532 non-null  datetime64[us]
 10  OUTTIME         61522 non-null  datetime64[us]
 11  LOS             61522 non-null  float64       
dtypes: Int64(6), datetime64[us](2), float64(1), object(3)
memory usage: 6.0+ MB


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
0,21891,17239,156279,203333,both,CCU,CCU,7,7,2133-10-23 15:19:16,2133-10-29 21:27:49,6.2559
1,35298,27810,118521,299036,both,CCU,CCU,7,7,2122-08-07 14:35:31,2122-08-09 22:45:29,2.3403
2,39733,31454,117800,209490,both,CCU,CCU,7,7,2143-08-21 00:20:52,2143-08-24 14:28:40,3.5888
3,37755,29805,196011,212996,both,CCU,CCU,7,7,2154-07-02 12:09:06,2154-07-08 14:23:46,6.0935
4,34046,26849,199270,298791,both,CCU,CCU,7,7,2136-08-01 16:35:56,2136-08-04 18:50:08,3.0932
5,41241,32739,147621,294198,both,CCU,CCU,7,7,2186-08-12 12:11:41,2186-08-30 18:53:52,18.2793
6,41242,32739,147621,273998,both,CCU,CCU,7,7,2186-09-05 04:04:39,2186-09-07 13:07:40,2.3771
7,41326,32805,169344,251001,both,CCU,CCU,7,7,2183-02-05 18:54:04,2183-02-06 16:47:32,0.9121
8,39805,31515,152958,200269,both,CCU,CCU,7,7,2102-06-29 23:15:05,2102-07-01 22:20:04,1.9618
9,41709,41143,112156,234585,both,CCU,CCU,7,7,2132-10-18 01:22:36,2132-10-26 21:47:01,8.8503


In [13]:
# aggregate icustays dataframe for meaningful data
icustays_agg_df = icustays_df.groupby('HADM_ID').agg(
    LONGEST_STAY=('LOS', 'max'),     # max length of stay
    SHORTEST_STAY=('LOS', 'min'),    # min length of stay
    TOTAL_ICUSTAYS=('ICUSTAY_ID', 'count')  # number of ICU stays
)

In [14]:
# review range of total icustays
icustays_agg_df['TOTAL_ICUSTAYS'].agg(['min', 'max'])

,TOTAL_ICUSTAYS
min,1
max,7


In [15]:
# review counts of number of icustays
icustays_agg_df['TOTAL_ICUSTAYS'].value_counts()

,count
TOTAL_ICUSTAYS,
1,54526
2,2865
3,326
4,54
5,10
6,3
7,2


SUPPLEMENTARY DATASETS

D_ITEM DATASET

In [16]:
# SQL query to select d_items from dataset - dataset used to understand clinical codes better
d_items_query = f"""
SELECT *
FROM `physionet-data.mimiciii_clinical.d_items`
"""

In [17]:
# load d_items dataframe to memory for reference later
d_items_df = load_file_results_to_df(query=d_items_query, bucket=gcs_bucket, client=bq_client)

Query executed and loaded to DataFrame using BigQuery
DataFrame with 12487 row(s) has been returned.


D_ICD_DIAGNOSES DATASETS

In [18]:
# SQL query to select d_icd_diagnoses from dataset - dataset used to understand clinical codes better
d_icd_diagnoses_query = f"""
SELECT *
FROM `physionet-data.mimiciii_clinical.d_icd_diagnoses`
"""

In [19]:
# load d_icd_diagnoses dataframe to memory for reference later
d_icd_diagnoses_df = load_file_results_to_df(query=d_icd_diagnoses_query, bucket=gcs_bucket, client=bq_client)

Query executed and loaded to DataFrame using BigQuery
DataFrame with 14567 row(s) has been returned.


D_ICD_PROCEDURES DATASETS

In [20]:
# SQL query to select d_icd_procedures from dataset - dataset used to understand clinical codes better
d_icd_procedures_query = f"""
SELECT *
FROM `physionet-data.mimiciii_clinical.d_icd_procedures`
"""

In [21]:
# load d_icd_procedures dataframe to memory for reference later
d_icd_procedures_df = load_file_results_to_df(query=d_icd_procedures_query, bucket=gcs_bucket, client=bq_client)

Query executed and loaded to DataFrame using BigQuery
DataFrame with 3882 row(s) has been returned.


D_LABITEMS DATASETS

In [22]:
# SQL query to select d_labitems from dataset - dataset used to understand clinical codes better
d_labitems_query = f"""
SELECT *
FROM `physionet-data.mimiciii_clinical.d_labitems`
"""

In [23]:
# load d_lab_events dataframe to memory for reference later
d_labitems_df = load_file_results_to_df(query=d_labitems_query, bucket=gcs_bucket, client=bq_client)

Query executed and loaded to DataFrame using BigQuery
DataFrame with 753 row(s) has been returned.


CLINICAL DATASETS

CHARTEVENTS DATASET

In [24]:
# SQL query to select relevant events from entire chartevents table
chartevents_query = f"""
SELECT
  COUNT(*) AS TOTAL_COUNT,
  COUNTIF(t.VALUENUM IS NULL) as VALUENUM_NULL
FROM `physionet-data.mimiciii_clinical.chartevents` t
ORDER BY
  VALUENUM_NULL DESC
"""

In [25]:
# run query and store dataframe to memory
chartevents_df = load_file_results_to_df(query= chartevents_query, bucket=gcs_bucket, client=bq_client)
chartevents_df

Query executed and loaded to DataFrame using BigQuery
DataFrame with 1 row(s) has been returned.


,TOTAL_COUNT,VALUENUM_NULL
0,330712483,184208315


In [26]:
# SQL query to select relevant events from entire chartevents table
chartevents_query = f"""
SELECT
  t.ITEMID AS ITEMID,
  COUNT(*) AS ITEMID_COUNT,
  COUNT(DISTINCT t.SUBJECT_ID) AS PATIENT_COUNT
FROM `physionet-data.mimiciii_clinical.chartevents` t
GROUP BY
  ITEMID
ORDER BY
  PATIENT_COUNT DESC
"""

In [27]:
# load filtered chartevents dataframe to memory
chartevents_by_itemid_df = load_file_results_to_df(query=chartevents_query, bucket=gcs_bucket, client=bq_client)

# merging clinical codes to their written references for understanding
chartevents_by_itemid_df = chartevents_by_itemid_df.merge(d_items_df[['ITEMID', 'LABEL']], on='ITEMID', how='left')

Query executed and loaded to DataFrame using BigQuery
DataFrame with 6463 row(s) has been returned.


In [30]:
pd.set_option('display.max_rows', None)
chartevents_by_itemid_df

,ITEMID,ITEMID_COUNT,PATIENT_COUNT,LABEL
0,926,49947,30599,Religion
1,211,5180809,29899,Heart Rate
2,31,1442406,28941,Activity
3,80,1136214,26156,Bowel Sounds
4,467,1155571,23480,O2 Delivery Device
5,919,41704,22748,Service
6,924,41704,22748,Readmission
7,930,41704,22748,Ext
8,925,41704,22748,Marital Status
9,69,41704,22748,BSA


In [ ]:
pd.reset_option('display.max_rows')

DATETIMEEVENTS DATASET ANALYSIS

In [ ]:
# SQL query to select datetimemevents
datetimeevents_query = f"""
SELECT *
FROM `physionet-data.mimiciii_clinical.datetimeevents`
"""

In [ ]:
# EXPLORE DATETIMEEVENTS DATAFRAME
datetimeevents_df = load_file_results_to_df(query=datetimeevents_query, bucket=gcs_bucket, client=bq_client)

Query executed and loaded to DataFrame using BigQuery
DataFrame with 4485937 row(s) has been returned.


In [ ]:
# check for duplicate patients
duplicate_subject_id_count = datetimeevents_df.duplicated(subset=['SUBJECT_ID']).sum()
print(f"Number of duplicate subject IDs is: {duplicate_subject_id_count}")

# check for duplicate admissions
duplicate_hadm_id_count = datetimeevents_df.duplicated(subset=['HADM_ID']).sum()
print(f"Number of duplicate admission IDs is: {duplicate_hadm_id_count}")

# reviewing missingness across the admissions dataset
datetimeevents_null_values_df = datetimeevents_df.isnull().sum().reset_index()
datetimeevents_null_values_df.columns = ['Column', 'Missing Count']
datetimeevents_null_values_df.sort_values(by='Missing Count', ascending=False)

Number of duplicate subject IDs is: 4456752
Number of duplicate admission IDs is: 4449718


,Column,Missing Count
12,RESULTSTATUS,4485937
13,STOPPED,2686920
11,ERROR,1799017
10,WARNING,1799017
8,VALUE,52090
3,ICUSTAY_ID,4156
2,HADM_ID,1772
4,ITEMID,0
0,ROW_ID,0
1,SUBJECT_ID,0


LABEVENTS DATASET

In [ ]:
# SQL query to select labevents
labevents_query = f"""
SELECT *
FROM `physionet-data.mimiciii_clinical.labevents`
"""

In [ ]:
labevents_df = load_file_results_to_df(query=labevents_query, bucket=gcs_bucket, client=bq_client)
labevents_df.info()
labevents_df.describe()
labevents_df.head(10)

Query executed and loaded to DataFrame using BigQuery
DataFrame with 27854055 row(s) has been returned.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27854055 entries, 0 to 27854054
Data columns (total 9 columns):
 #   Column      Dtype         
---  ------      -----         
 0   ROW_ID      Int64         
 1   SUBJECT_ID  Int64         
 2   HADM_ID     Int64         
 3   ITEMID      Int64         
 4   CHARTTIME   datetime64[us]
 5   VALUE       object        
 6   VALUENUM    float64       
 7   VALUEUOM    object        
 8   FLAG        object        
dtypes: Int64(4), datetime64[us](1), float64(1), object(3)
memory usage: 2.0+ GB


,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG
0,27632053,98130,<NA>,51506,2189-07-21 06:45:00,Hazy,NaN,None,None
1,24995233,77645,<NA>,51506,2184-06-16 11:45:00,Clear,NaN,None,None
2,19992285,40601,182879,51506,2184-08-04 01:55:00,Clear,NaN,None,None
3,5960883,9505,<NA>,51506,2188-10-29 14:30:00,Clear,NaN,None,None
4,1500641,2365,<NA>,50995,2178-11-03 14:30:00,1.1,1.1,ng/dL,None
5,359629,518,120954,51251,2109-03-26 11:05:00,0,0.0,%,None
6,14980294,24248,<NA>,51251,2161-09-08 12:40:00,0,0.0,%,None
7,11156945,18014,134816,51251,2177-04-18 03:30:00,0,0.0,%,None
8,25668205,82405,189378,51251,2123-06-14 23:00:00,3,3.0,%,abnormal
9,18884314,30731,184324,51251,2141-11-09 02:56:00,5,5.0,%,abnormal


In [ ]:
# reviewing missingness across the labevents dataset
labevents_null_values_df = labevents_df.isnull().sum().reset_index()
labevents_null_values_df.columns = ['Column', 'Missing Count']
labevents_null_values_df.sort_values(by='Missing Count', ascending=False)

# Count the number of lab events per HADM_ID
lab_events_per_hadm = labevents_df.groupby('HADM_ID').size()

# Calculate the average number of lab events per HADM_ID
average_lab_events = lab_events_per_hadm.mean()
print('Average number of lab events per HADM_ID: ' + str(average_lab_events))

# Calculate the average number of lab events per HADM_ID
min_lab_events = lab_events_per_hadm.min()
print('Min number of lab events per HADM_ID:' + str(min_lab_events))

# Calculate the average number of lab events per HADM_ID
max_lab_events = lab_events_per_hadm.max()
print('Max number of lab events per HADM_ID:' + str(max_lab_events))

Average number of lab events per HADM_ID: 382.5391480799986
Min number of lab events per HADM_ID:1
Max number of lab events per HADM_ID:13713


In [ ]:
# merging clinical codes to their written references for understanding
merged_labevents_ref_df = labevents_df.merge(d_labitems_df[['ITEMID', 'LABEL']], on='ITEMID', how='left')

# count the occurrences of each labevent by itemid
labevents_count_df = merged_labevents_ref_df.groupby(['ITEMID', 'LABEL']).size().reset_index(name='COUNT')
labevents_count_df

,ITEMID,LABEL,COUNT
0,50800,SPECIMEN TYPE,404785
1,50801,Alveolar-arterial Gradient,22016
2,50802,Base Excess,490651
3,50803,"Calculated Bicarbonate, Whole Blood",9246
4,50804,Calculated Total CO2,490641
...,...,...,...
721,51534,MYELOS,2
722,51535,CD55,1
723,51536,CD59,1
724,51537,TDT,1


In [ ]:
# get the top lab events
top_lab_events = labevents_count_df.head(20)
top_lab_events

,ITEMID,LABEL,COUNT
0,50800,SPECIMEN TYPE,404785
1,50801,Alveolar-arterial Gradient,22016
2,50802,Base Excess,490651
3,50803,"Calculated Bicarbonate, Whole Blood",9246
4,50804,Calculated Total CO2,490641
5,50805,Carboxyhemoglobin,2056
6,50806,"Chloride, Whole Blood",48188
7,50808,Free Calcium,249110
8,50809,Glucose,196736
9,50810,"Hematocrit, Calculated",89715


MICROBIOLOGY EVENTS DATASET



In [ ]:
# SQL query to select microbiologyevents
microbiologyevents_query = f"""
SELECT *
FROM `physionet-data.mimiciii_clinical.microbiologyevents`
"""

In [ ]:
# load microbiologyevents dataframe to memory
microbiologyevents_df = load_file_results_to_df(query=microbiologyevents_query, bucket=gcs_bucket, client=bq_client)

Query executed and loaded to DataFrame using BigQuery
DataFrame with 631726 row(s) has been returned.


In [ ]:
microbiologyevents_df.info()
microbiologyevents_df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 631726 entries, 0 to 631725
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   ROW_ID               631726 non-null  Int64         
 1   SUBJECT_ID           631726 non-null  Int64         
 2   HADM_ID              631726 non-null  Int64         
 3   CHARTDATE            631726 non-null  datetime64[us]
 4   CHARTTIME            589954 non-null  datetime64[us]
 5   SPEC_ITEMID          631647 non-null  Int64         
 6   SPEC_TYPE_DESC       631726 non-null  object        
 7   ORG_ITEMID           326881 non-null  Int64         
 8   ORG_NAME             328016 non-null  object        
 9   ISOLATE_NUM          328016 non-null  Int64         
 10  AB_ITEMID            275834 non-null  Int64         
 11  AB_NAME              275834 non-null  object        
 12  DILUTION_TEXT        267350 non-null  object        
 13  DILUTION_COMPA

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,SPEC_ITEMID,SPEC_TYPE_DESC,ORG_ITEMID,ORG_NAME,ISOLATE_NUM,AB_ITEMID,AB_NAME,DILUTION_TEXT,DILUTION_COMPARISON,DILUTION_VALUE,INTERPRETATION
0,428564,40822,102495,2103-07-19,2103-07-19 10:43:00,<NA>,Touch Prep/Sections,<NA>,None,<NA>,<NA>,None,None,None,<NA>,None
1,481195,55801,165940,2159-05-02,2159-05-02 15:52:00,<NA>,Touch Prep/Sections,<NA>,None,<NA>,<NA>,None,None,None,<NA>,None
2,539558,72908,176846,2128-01-25,2128-01-25 12:00:00,<NA>,Touch Prep/Sections,<NA>,None,<NA>,<NA>,None,None,None,<NA>,None
3,539806,72978,198761,2189-05-21,2189-05-21 12:29:00,<NA>,Touch Prep/Sections,<NA>,None,<NA>,<NA>,None,None,None,<NA>,None
4,552049,76600,197900,2103-06-26,2103-06-26 10:00:00,<NA>,POST-MORTEM VIRAL CULTURE,<NA>,None,<NA>,<NA>,None,None,None,<NA>,None
5,501020,61296,188877,2154-09-16,2154-09-16 09:26:00,<NA>,Touch Prep/Sections,<NA>,None,<NA>,<NA>,None,None,None,<NA>,None
6,522304,67717,168794,2201-06-29,2201-06-29 10:12:00,<NA>,Touch Prep/Sections,<NA>,None,<NA>,<NA>,None,None,None,<NA>,None
7,518565,66534,159220,2173-05-28,2173-05-28 13:35:00,<NA>,Touch Prep/Sections,<NA>,None,<NA>,<NA>,None,None,None,<NA>,None
8,592727,88782,123350,2101-11-14,NaT,<NA>,"C, E, & A Screening",<NA>,None,<NA>,<NA>,None,None,None,<NA>,None
9,384674,29035,154213,2156-10-14,NaT,<NA>,Isolate,80004,KLEBSIELLA PNEUMONIAE,1,<NA>,None,None,None,<NA>,None


In [ ]:
# check for duplicate patients
duplicate_subject_id_count = microbiologyevents_df.duplicated(subset=['SUBJECT_ID']).sum()
print(f"Number of duplicate subject IDs is: {duplicate_subject_id_count}")

# check for duplicate admissions
duplicate_hadm_id_count = microbiologyevents_df.duplicated(subset=['HADM_ID']).sum()
print(f"Number of duplicate admission IDs is: {duplicate_hadm_id_count}")

# reviewing missingness across the microbiologyevents dataset
microbiologyevents_null_values_df = microbiologyevents_df.isnull().sum().reset_index()
microbiologyevents_null_values_df.columns = ['Column', 'Missing Count']
microbiologyevents_null_values_df.sort_values(by='Missing Count', ascending=False)

Number of duplicate subject IDs is: 592542
Number of duplicate admission IDs is: 582986


,Column,Missing Count
13,DILUTION_COMPARISON,364548
14,DILUTION_VALUE,364548
12,DILUTION_TEXT,364376
11,AB_NAME,355892
15,INTERPRETATION,355892
10,AB_ITEMID,355892
7,ORG_ITEMID,304845
8,ORG_NAME,303710
9,ISOLATE_NUM,303710
4,CHARTTIME,41772


OUTPUTEVENTS DATASET

In [ ]:
# SQL query to select outputevents
outputevents_query = f"""
SELECT *
FROM `physionet-data.mimiciii_clinical.outputevents`
"""

In [ ]:
# load outputevents dataframe to memory
outputevents_df = load_file_results_to_df(query=outputevents_query, bucket=gcs_bucket, client=bq_client)

Query executed and loaded to DataFrame using BigQuery
DataFrame with 4349218 row(s) has been returned.


In [ ]:
outputevents_df.head(10)

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR
0,4013738,73472,114297,286892,2157-09-17 06:00:00,226560,400.0,mL,2157-09-17 06:49:00,19710,None,None,None
1,4013736,73472,114297,286892,2157-09-16 15:00:00,226560,300.0,mL,2157-09-16 17:09:00,18147,None,None,None
2,4013730,73472,114297,286892,2157-09-16 04:00:00,226560,275.0,mL,2157-09-16 04:19:00,18462,None,None,None
3,3927312,74496,138943,285604,2142-06-29 03:00:00,226560,225.0,mL,2142-06-29 02:37:00,17058,None,None,None
4,3998294,76800,182372,267744,2110-09-15 06:00:00,226560,200.0,mL,2110-09-15 06:39:00,17806,None,None,None
5,3998277,76800,182372,267744,2110-09-12 04:00:00,226560,200.0,mL,2110-09-12 04:17:00,14228,None,None,None
6,3695716,78336,186636,265554,2168-11-05 06:47:00,226560,700.0,mL,2168-11-05 06:47:00,16870,None,None,None
7,4345757,88064,107776,257016,2100-08-09 15:30:00,226560,450.0,mL,2100-08-09 15:31:00,18576,None,None,None
8,3920622,68353,106149,240821,2168-07-21 02:00:00,226560,150.0,mL,2168-07-21 02:09:00,16934,None,None,None
9,4023533,75265,128105,208370,2183-11-11 23:00:00,226560,200.0,mL,2183-11-12 02:26:00,16797,None,None,None


In [ ]:
# merge with d_items reference dataset
joined_outputevents_d_items_df = outputevents_df.merge(d_items_df[['ITEMID', 'LABEL']], on='ITEMID', how='left')

In [ ]:
joined_outputevents_d_items_df.head(10)

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL
0,4013738,73472,114297,286892,2157-09-17 06:00:00,226560,400.0,mL,2157-09-17 06:49:00,19710,None,None,None,Void
1,4013736,73472,114297,286892,2157-09-16 15:00:00,226560,300.0,mL,2157-09-16 17:09:00,18147,None,None,None,Void
2,4013730,73472,114297,286892,2157-09-16 04:00:00,226560,275.0,mL,2157-09-16 04:19:00,18462,None,None,None,Void
3,3927312,74496,138943,285604,2142-06-29 03:00:00,226560,225.0,mL,2142-06-29 02:37:00,17058,None,None,None,Void
4,3998294,76800,182372,267744,2110-09-15 06:00:00,226560,200.0,mL,2110-09-15 06:39:00,17806,None,None,None,Void
5,3998277,76800,182372,267744,2110-09-12 04:00:00,226560,200.0,mL,2110-09-12 04:17:00,14228,None,None,None,Void
6,3695716,78336,186636,265554,2168-11-05 06:47:00,226560,700.0,mL,2168-11-05 06:47:00,16870,None,None,None,Void
7,4345757,88064,107776,257016,2100-08-09 15:30:00,226560,450.0,mL,2100-08-09 15:31:00,18576,None,None,None,Void
8,3920622,68353,106149,240821,2168-07-21 02:00:00,226560,150.0,mL,2168-07-21 02:09:00,16934,None,None,None,Void
9,4023533,75265,128105,208370,2183-11-11 23:00:00,226560,200.0,mL,2183-11-12 02:26:00,16797,None,None,None,Void


In [ ]:
joined_outputevents_d_items_df.info()
joined_outputevents_d_items_df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4349218 entries, 0 to 4349217
Data columns (total 14 columns):
 #   Column      Dtype         
---  ------      -----         
 0   ROW_ID      Int64         
 1   SUBJECT_ID  Int64         
 2   HADM_ID     Int64         
 3   ICUSTAY_ID  Int64         
 4   CHARTTIME   datetime64[us]
 5   ITEMID      Int64         
 6   VALUE       float64       
 7   VALUEUOM    object        
 8   STORETIME   datetime64[us]
 9   CGID        Int64         
 10  STOPPED     object        
 11  NEWBOTTLE   object        
 12  ISERROR     object        
 13  LABEL       object        
dtypes: Int64(6), datetime64[us](2), float64(1), object(5)
memory usage: 489.4+ MB


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR,LABEL
0,4013738,73472,114297,286892,2157-09-17 06:00:00,226560,400.0,mL,2157-09-17 06:49:00,19710,None,None,None,Void
1,4013736,73472,114297,286892,2157-09-16 15:00:00,226560,300.0,mL,2157-09-16 17:09:00,18147,None,None,None,Void
2,4013730,73472,114297,286892,2157-09-16 04:00:00,226560,275.0,mL,2157-09-16 04:19:00,18462,None,None,None,Void
3,3927312,74496,138943,285604,2142-06-29 03:00:00,226560,225.0,mL,2142-06-29 02:37:00,17058,None,None,None,Void
4,3998294,76800,182372,267744,2110-09-15 06:00:00,226560,200.0,mL,2110-09-15 06:39:00,17806,None,None,None,Void
5,3998277,76800,182372,267744,2110-09-12 04:00:00,226560,200.0,mL,2110-09-12 04:17:00,14228,None,None,None,Void
6,3695716,78336,186636,265554,2168-11-05 06:47:00,226560,700.0,mL,2168-11-05 06:47:00,16870,None,None,None,Void
7,4345757,88064,107776,257016,2100-08-09 15:30:00,226560,450.0,mL,2100-08-09 15:31:00,18576,None,None,None,Void
8,3920622,68353,106149,240821,2168-07-21 02:00:00,226560,150.0,mL,2168-07-21 02:09:00,16934,None,None,None,Void
9,4023533,75265,128105,208370,2183-11-11 23:00:00,226560,200.0,mL,2183-11-12 02:26:00,16797,None,None,None,Void


In [ ]:
# check for duplicate patients
duplicate_subject_id_count = joined_outputevents_d_items_df.duplicated(subset=['SUBJECT_ID']).sum()
print(f"Number of duplicate subject IDs is: {duplicate_subject_id_count}")

# check for duplicate admissions
duplicate_hadm_id_count = joined_outputevents_d_items_df.duplicated(subset=['HADM_ID']).sum()
print(f"Number of duplicate admission IDs is: {duplicate_hadm_id_count}")

# reviewing missingness across the microbiologyevents dataset
outputevents_null_values_df = joined_outputevents_d_items_df.isnull().sum().reset_index()
outputevents_null_values_df.columns = ['Column', 'Missing Count']
outputevents_null_values_df.sort_values(by='Missing Count', ascending=False)

Number of duplicate subject IDs is: 4307361
Number of duplicate admission IDs is: 4297209


,Column,Missing Count
12,ISERROR,4349218
11,NEWBOTTLE,4349218
10,STOPPED,4349218
6,VALUE,88549
7,VALUEUOM,36251
3,ICUSTAY_ID,7190
2,HADM_ID,4839
4,CHARTTIME,0
0,ROW_ID,0
1,SUBJECT_ID,0


In [ ]:
# count the occurrences of each labevent by itemid
outputevents_count_df = joined_outputevents_d_items_df.groupby(['ITEMID', 'LABEL']).size().reset_index(name='COUNT')
outputevents_count_df.head(20)

,ITEMID,LABEL,COUNT
0,40048,Chest Tubes Left Pleural 1,24144
1,40049,Chest Tubes Right Pleural 1,25448
2,40050,Chest Tubes Right Pleural 2,5361
3,40051,Gastric Gastric Tube,7624
4,40052,Gastric Nasogastric,24726
5,40053,Stool Out Fecal Bag,4884
6,40054,Stool Out Stool,81828
7,40055,Urine Out Foley,1917421
8,40056,Urine Out Lt Nephrostomy,2726
9,40057,Urine Out Rt Nephrostomy,2797


In [ ]:
# item_ids = [211, 829, 646, 813, 618, 811, 184, 723, 454, 198, 837, 791, 781, 828, 788, 787, 87, 861, 814, 833, 821, 677, 455, 8441]

DIAGNOSES, MEDICATIONS AND PROCEDURES DATASETS

DIAGNOSES DATASET ANALYSIS


In [ ]:
# SQL query to select procedures based on subject_id
diagnoses_icd_query = f"""
SELECT *
FROM `physionet-data.mimiciii_clinical.diagnoses_icd`
"""

In [ ]:
# load diagnoses_icd dataframe to memory
diagnoses_icd_df = load_file_results_to_df(query=diagnoses_icd_query, bucket=gcs_bucket, client=bq_client)

Query executed and loaded to DataFrame using BigQuery
DataFrame with 651047 row(s) has been returned.


In [ ]:
diagnoses_icd_df.info()
diagnoses_icd_df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651047 entries, 0 to 651046
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   ROW_ID      651047 non-null  Int64 
 1   SUBJECT_ID  651047 non-null  Int64 
 2   HADM_ID     651047 non-null  Int64 
 3   SEQ_NUM     651000 non-null  Int64 
 4   ICD9_CODE   651000 non-null  object
dtypes: Int64(4), object(1)
memory usage: 27.3+ MB


,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,3113,256,108811,1,53240
1,3114,256,108811,2,41071
2,3115,256,108811,3,53560
3,3116,256,108811,4,40390
4,3117,256,108811,5,5859
5,3118,256,108811,6,04186
6,3119,256,108811,7,41401
7,3120,256,108811,8,2724
8,3121,256,108811,9,496
9,3122,256,108811,10,42731


In [ ]:
diagnoses_icd_df.describe()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM
count,651047.0,651047.0,651047.0,651000.0
mean,325524.0,38971.159758,150017.744819,7.913836
std,187941.22469,29372.198841,28878.068648,6.072633
min,1.0,2.0,100001.0,1.0
25%,162762.5,14562.5,125028.0,3.0
50%,325524.0,28671.0,150140.0,6.0
75%,488285.5,63715.0,174978.0,11.0
max,651047.0,99999.0,199999.0,39.0


In [ ]:
# reviewing missingness across the admissions dataset
diagnoses_null_values_df = diagnoses_icd_df.isnull().sum().reset_index()
diagnoses_null_values_df.columns = ['Column', 'Missing Count']
diagnoses_null_values_df.sort_values(by='Missing Count', ascending=False)

,Column,Missing Count
4,ICD9_CODE,47
3,SEQ_NUM,47
0,ROW_ID,0
2,HADM_ID,0
1,SUBJECT_ID,0


In [ ]:
# merging clinical codes to their written references for understanding
merged_diagnoses_ref_df = diagnoses_icd_df.merge(d_icd_diagnoses_df[['ICD9_CODE', 'SHORT_TITLE']], on='ICD9_CODE', how='left')

# count the occurrences of each labevent by itemid
diagnoses_count_df = merged_diagnoses_ref_df.groupby(['ICD9_CODE', 'SHORT_TITLE']).size().reset_index(name='COUNT')
diagnoses_count_df.sort_values(by='COUNT', ascending=False)

,ICD9_CODE,SHORT_TITLE,COUNT
1920,4019,Hypertension NOS,20703
2066,4280,CHF NOS,13111
2055,42731,Atrial fibrillation,12891
1977,41401,Crnry athrscl natve vssl,12429
2897,5849,Acute kidney failure NOS,9119
...,...,...,...
5320,8902,Opn wnd hip/thigh w tend,1
5317,8875,"Amput arm, unil NOS-comp",1
6837,V9039,Retain organic frag NEC,1
6836,V902,Retain plastic fragments,1


INPUTEVENTSMV DATASET ANALYSIS

In [ ]:
inputevents_mv_query = """
SELECT *
FROM `physionet-data.mimiciii_clinical.inputevents_mv`
"""

In [ ]:
inputevents_mv_df = load_file_results_to_df(query=inputevents_mv_query, bucket=gcs_bucket, client=bq_client)
inputevents_mv_df.head()
inputevents_mv_df.info()
inputevents_mv_df.describe()

Query executed and loaded to DataFrame using BigQuery
DataFrame with 3618991 row(s) has been returned.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3618991 entries, 0 to 3618990
Data columns (total 31 columns):
 #   Column                         Dtype         
---  ------                         -----         
 0   ROW_ID                         Int64         
 1   SUBJECT_ID                     Int64         
 2   HADM_ID                        Int64         
 3   ICUSTAY_ID                     Int64         
 4   STARTTIME                      datetime64[us]
 5   ENDTIME                        datetime64[us]
 6   ITEMID                         Int64         
 7   AMOUNT                         float64       
 8   AMOUNTUOM                      object        
 9   RATE                           float64       
 10  RATEUOM                        object        
 11  STORETIME                      datetime64[us]
 12  CGID                           Int64         
 13  ORDERID       

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,ITEMID,AMOUNT,RATE,STORETIME,...,ORDERID,LINKORDERID,PATIENTWEIGHT,TOTALAMOUNT,ISOPENBAG,CONTINUEINNEXTDEPT,CANCELREASON,COMMENTS_DATE,ORIGINALAMOUNT,ORIGINALRATE
count,3618991.0,3618991.0,3618991.0,3617827.0,3618991,3618991,3618991.0,3.618991e+06,2.042852e+06,3618991,...,3618991.0,3618991.0,3.618991e+06,3.084964e+06,3618991.0,3618991.0,3618991.0,368625,3.618991e+06,3.618991e+06
mean,1809496.0,63578.306407,150125.515375,249684.097579,2151-12-02 07:09:11.502129,2151-12-02 10:07:08.358035,224245.453668,2.189589e+02,8.941301e+01,2151-12-02 08:55:45.047078,...,4999339.874992,5002586.999459,8.555881e+01,2.548489e+02,0.000444,0.000023,0.170734,2151-11-23 20:08:42.507099,3.414276e+02,8.725674e+01
min,1.0,23.0,100001.0,200001.0,2100-06-08 02:36:00,2100-06-08 04:41:00,220862.0,-5.250000e+05,-9.060000e+02,2100-06-08 02:36:00,...,2.0,2.0,1.000000e+00,2.000000e-03,0.0,0.0,0.0,2100-07-07 04:28:00,-1.058330e+02,-8.785455e+02
25%,904748.5,48872.0,125521.0,224593.0,2126-10-04 03:55:00,2126-10-04 08:30:30,221906.0,2.000000e+00,4.012852e+00,2126-10-04 08:00:00,...,2499702.0,2497315.0,6.840000e+01,1.000000e+02,0.0,0.0,0.0,2126-07-13 15:42:00,1.000000e+01,1.000000e+00
50%,1809496.0,65370.0,150324.0,250285.0,2152-03-29 19:45:00,2152-03-30 01:43:00,225158.0,2.798783e+01,1.800720e+01,2152-03-30 00:07:00,...,4997277.0,4999979.0,8.140000e+01,1.000000e+02,0.0,0.0,0.0,2152-02-14 04:03:00,6.076326e+01,1.000000e+01
75%,2714243.5,82065.0,175201.0,274326.0,2177-03-15 02:03:30,2177-03-15 04:18:00,225942.0,1.000000e+02,5.000000e+01,2177-03-15 02:38:00,...,7500282.0,7506212.0,9.800000e+01,2.500000e+02,0.0,0.0,0.0,2177-07-04 10:02:00,2.000000e+02,5.000000e+01
max,3618991.0,99999.0,199999.0,299998.0,2209-08-07 15:41:00,2209-08-07 16:51:00,228383.0,1.000150e+06,2.424000e+06,2209-08-07 15:41:00,...,10000000.0,10000000.0,8.106000e+03,1.000150e+06,1.0,1.0,2.0,2209-08-07 11:27:00,9.999999e+06,2.424000e+06
std,1044712.85836,23249.788192,28704.11204,28803.860164,NaN,NaN,2138.378228,1.433101e+03,3.311696e+03,NaN,...,2886804.026296,2888102.268832,3.132838e+01,9.284953e+02,0.021061,0.004846,0.528522,NaN,7.675057e+03,2.605462e+03


In [ ]:
# reviewing missingness across the admissions dataset
inputevents_null_values_df = inputevents_mv_df.isnull().sum().reset_index()
inputevents_null_values_df.columns = ['Column', 'Missing Count']
inputevents_null_values_df.sort_values(by='Missing Count', ascending=False)

,Column,Missing Count
27,COMMENTS_CANCELEDBY,3499627
26,COMMENTS_EDITEDBY,3369730
28,COMMENTS_DATE,3250366
9,RATE,1576139
10,RATEUOM,1576139
16,SECONDARYORDERCATEGORYNAME,983428
20,TOTALAMOUNT,534027
21,TOTALAMOUNTUOM,530354
3,ICUSTAY_ID,1164
6,ITEMID,0


In [ ]:
# inputevents query for understanding counts across itemid and related patient count for each item
inputevents_mv_query = """
SELECT
  itemid AS ITEMID,
  COUNT(*) AS ITEMID_COUNT,
  COUNT(DISTINCT subject_id) AS PATIENT_COUNT
FROM `physionet-data.mimiciii_clinical.inputevents_mv`
GROUP BY
  itemid
"""

In [ ]:
# run query
inputevents_mv_df = load_file_results_to_df(query=inputevents_mv_query, bucket=gcs_bucket, client=bq_client)

Query executed and loaded to DataFrame using BigQuery
DataFrame with 278 row(s) has been returned.


In [ ]:
# merge against item id for interpretation
joined_inputevents_mv_d_item_df = inputevents_mv_df.merge(d_items_df[['ITEMID', 'LABEL']], on='ITEMID', how='left')

In [ ]:
# print top inputevents for understanding
joined_inputevents_mv_d_item_df.sort_values(by='ITEMID_COUNT', ascending=False).head(100)

,ITEMID,ITEMID_COUNT,PATIENT_COUNT,LABEL
31,225158,527855,15898,NaCl 0.9%
1,220949,406345,13866,Dextrose 5%
36,225943,246312,11201,Solution
49,222168,178819,7255,Propofol
218,226452,135438,13365,PO Intake
...,...,...,...,...
122,225879,3279,1403,Levofloxacin
152,225911,3134,1684,Ranitidine (Prophylaxis)
129,225881,2998,382,Linezolid
29,225155,2929,375,Octreotide


In [ ]:
joined_inputevents_mv_d_item_df.sort_values(by='PATIENT_COUNT', ascending=False).head(100)

,ITEMID,ITEMID_COUNT,PATIENT_COUNT,LABEL
31,225158,527855,15898,NaCl 0.9%
1,220949,406345,13866,Dextrose 5%
218,226452,135438,13365,PO Intake
36,225943,246312,11201,Solution
252,222011,25897,9281,Magnesium Sulfate
...,...,...,...,...
170,226880,10628,469,Nutren Pulmonary (Full)
121,225875,1642,465,Gentamicin
41,227525,15636,455,Calcium Gluconate (CRRT)
174,227695,9853,447,Fibersource HN (Full)


INPUTEVENTSCV DATASET ANALYSIS

In [ ]:
inputevents_cv_query = """
SELECT *
FROM `physionet-data.mimiciii_clinical.inputevents_cv`
WHERE AMOUNT IS NOT NULL
  AND ITEMID IS NOT NULL
"""

In [ ]:
inputevents_cv_df = load_file_results_to_df(query=inputevents_cv_query, bucket=gcs_bucket, client=bq_client)
inputevents_cv_df.head(10)
inputevents_cv_df.info()

Query executed and loaded to DataFrame using BigQuery
DataFrame with 12422557 row(s) has been returned.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12422557 entries, 0 to 12422556
Data columns (total 22 columns):
 #   Column             Dtype         
---  ------             -----         
 0   ROW_ID             Int64         
 1   SUBJECT_ID         Int64         
 2   HADM_ID            Int64         
 3   ICUSTAY_ID         Int64         
 4   CHARTTIME          datetime64[us]
 5   ITEMID             Int64         
 6   AMOUNT             float64       
 7   AMOUNTUOM          object        
 8   RATE               float64       
 9   RATEUOM            object        
 10  STORETIME          datetime64[us]
 11  CGID               Int64         
 12  ORDERID            Int64         
 13  LINKORDERID        Int64         
 14  STOPPED            object        
 15  NEWBOTTLE          Int64         
 16  ORIGINALAMOUNT     float64       
 17  ORIGINALAMOUNTUOM  object        
 

In [ ]:
# reviewing missingness across the admissions dataset
inputevents_null_values_df = inputevents_cv_df.isnull().sum().reset_index()
inputevents_null_values_df.columns = ['Column', 'Missing Count']
inputevents_null_values_df.sort_values(by='Missing Count', ascending=False)

,Column,Missing Count
8,RATE,12422557
9,RATEUOM,12422557
21,ORIGINALSITE,12259103
15,NEWBOTTLE,12234536
14,STOPPED,12073233
19,ORIGINALRATE,7354024
20,ORIGINALRATEUOM,7299467
16,ORIGINALAMOUNT,3314206
7,AMOUNTUOM,160279
17,ORIGINALAMOUNTUOM,102328


In [ ]:
# inputevents query for understanding counts across itemid and related patient count for each item
inputevents_cv_query = """
SELECT
  itemid AS ITEMID,
  COUNT(*) AS ITEMID_COUNT,
  COUNT(DISTINCT subject_id) AS PATIENT_COUNT
FROM `physionet-data.mimiciii_clinical.inputevents_cv`
GROUP BY
  ITEMID
"""

In [ ]:
# run query
inputevents_cv_df = load_file_results_to_df(query=inputevents_cv_query, bucket=gcs_bucket, client=bq_client)

Query executed and loaded to DataFrame using BigQuery
DataFrame with 2938 row(s) has been returned.


In [ ]:
# merge against item id for interpretation
joined_inputevents_cv_d_item_df = inputevents_cv_df.merge(d_items_df[['ITEMID', 'LABEL']], on='ITEMID', how='left')

In [ ]:
# print top inputevents for understanding
joined_inputevents_cv_d_item_df.sort_values(by='ITEMID_COUNT', ascending=False).head(100)

,ITEMID,ITEMID_COUNT,PATIENT_COUNT,LABEL
306,30013,2557507,21418,D5W
337,30018,2392372,18353,.9% Normal Saline
1143,30131,924614,10502,Propofol
519,30045,825758,6274,Insulin
379,30025,813242,5138,Heparin
...,...,...,...,...
1114,30127,14317,214,Neosynephrine
820,30086,13434,181,Ultracal
1535,30194,12788,197,TPN D9.0
610,30313,12481,136,TPN D14


In [ ]:
# print top inputevents for understanding
joined_inputevents_cv_d_item_df.sort_values(by='PATIENT_COUNT', ascending=False).head(100)

,ITEMID,ITEMID_COUNT,PATIENT_COUNT,LABEL
306,30013,2557507,21418,D5W
337,30018,2392372,18353,.9% Normal Saline
601,30056,188107,17500,Po Intake
389,30026,291669,11161,KCL
1143,30131,924614,10502,Propofol
...,...,...,...,...
1021,30369,5071,346,SC 20 Fe PO
1392,30172,51536,337,Natrecor
326,30272,8414,337,E 24 FS PO
550,30048,21685,320,Lidocaine


PRESCRIPTIONS DATASET ANALYSIS

In [ ]:
prescriptions_query = """
SELECT *
FROM `physionet-data.mimiciii_clinical.prescriptions`
"""
prescriptions_df = load_file_results_to_df(query=prescriptions_query, bucket=gcs_bucket, client=bq_client)

Query executed and loaded to DataFrame using BigQuery
DataFrame with 4156450 row(s) has been returned.


In [ ]:
prescriptions_df.info()
prescriptions_df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4156450 entries, 0 to 4156449
Data columns (total 19 columns):
 #   Column             Dtype         
---  ------             -----         
 0   ROW_ID             Int64         
 1   SUBJECT_ID         Int64         
 2   HADM_ID            Int64         
 3   ICUSTAY_ID         Int64         
 4   STARTDATE          datetime64[us]
 5   ENDDATE            datetime64[us]
 6   DRUG_TYPE          object        
 7   DRUG               object        
 8   DRUG_NAME_POE      object        
 9   DRUG_NAME_GENERIC  object        
 10  FORMULARY_DRUG_CD  object        
 11  GSN                object        
 12  NDC                Int64         
 13  PROD_STRENGTH      object        
 14  DOSE_VAL_RX        object        
 15  DOSE_UNIT_RX       object        
 16  FORM_VAL_DISP      object        
 17  FORM_UNIT_DISP     object        
 18  ROUTE              object        
dtypes: Int64(5), datetime64[us](2), object(12)
memory usage: 622.3+

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,1934054,12519,106575,<NA>,2195-04-27,2195-05-03,MAIN,Insulin Pump,Insulin Pump,None,None,None,<NA>,None,None,None,None,None,SC
1,4060998,54609,173864,221161,2173-02-27,NaT,MAIN,Albuterol 0.083% Neb Soln,Albuterol 0.083% Neb Soln,None,ALBU3H,None,<NA>,None,None,None,None,None,None
2,1054881,28102,191948,<NA>,2170-06-17,NaT,MAIN,Fentanyl Citrate,Fentanyl Citrate,None,FENT2I,None,<NA>,None,None,None,None,None,None
3,744158,94998,117228,294773,2128-07-01,NaT,MAIN,HYDROmorphone (Dilaudid),HYDROmorphone (Dilaudid),None,HYDR2I,None,<NA>,None,None,None,None,None,None
4,2744262,17279,105180,296633,2151-12-24,NaT,MAIN,Norepinephrine,Norepinephrine,None,LEVO4I,None,<NA>,None,None,None,None,None,None
5,57565,65126,141349,215664,2134-08-29,NaT,MAIN,Magnesium Sulfate,Magnesium Sulfate,None,MAG2PM,None,<NA>,None,None,None,None,None,None
6,1338605,68135,164088,<NA>,2160-07-11,NaT,MAIN,Magnesium Sulfate,Magnesium Sulfate,None,MAG2PM,None,<NA>,None,None,None,None,None,None
7,2213838,7888,106348,<NA>,2165-01-06,NaT,MAIN,MAGNESIUM SULFATE,MAGNESIUM SULFATE,None,MAGS1I,None,<NA>,None,None,None,None,None,None
8,813535,96731,127839,250955,2121-01-24,NaT,MAIN,Prochlorperazine,Prochlorperazine,None,COMP10I,None,<NA>,None,None,None,None,None,None
9,2934191,30825,143552,275309,2116-01-03,NaT,MAIN,Diazepam,Diazepam,None,DIAZ10I,None,<NA>,None,None,None,None,None,None


In [ ]:
prescriptions_counts = prescriptions_df.groupby('DRUG').agg(
    DRUG_COUNT = ('DRUG', 'count'),
    PATIENT_COUNT = ('SUBJECT_ID', 'nunique')
).reset_index()
pd.set_option('display.max_rows', None)
prescriptions_counts.sort_values(by='DRUG_COUNT', ascending=False)

,DRUG,DRUG_COUNT,PATIENT_COUNT
3051,Potassium Chloride,192993,29711
1848,Insulin,143465,25235
1036,D5W,142241,21279
1547,Furosemide,133122,19738
124,0.9% Sodium Chloride,130147,15026
2622,NS,129731,20390
2230,Magnesium Sulfate,90427,26120
1904,Iso-Osmotic Dextrose,87005,21098
3398,Sodium Chloride 0.9% Flush,83392,29387
195,Acetaminophen,78768,28821


PROCEDURESEVENTS_MV DATASET ANALYSIS

In [ ]:
# SQL query to extract procedure events from BigQuery
procedureevents_mv_query = f"""
SELECT *
FROM `physionet-data.mimiciii_clinical.procedureevents_mv`
"""

In [ ]:
procedureevents_mv_df = load_file_results_to_df(query=procedureevents_mv_query, bucket=gcs_bucket, client=bq_client)
procedureevents_mv_df.info()
procedureevents_mv_df.head(10)

Query executed and loaded to DataFrame using BigQuery
DataFrame with 258066 row(s) has been returned.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258066 entries, 0 to 258065
Data columns (total 25 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   ROW_ID                      258066 non-null  Int64         
 1   SUBJECT_ID                  258066 non-null  Int64         
 2   HADM_ID                     258066 non-null  Int64         
 3   ICUSTAY_ID                  257978 non-null  Int64         
 4   STARTTIME                   258066 non-null  datetime64[us]
 5   ENDTIME                     258066 non-null  datetime64[us]
 6   ITEMID                      258066 non-null  Int64         
 7   VALUE                       258066 non-null  float64       
 8   VALUEUOM                    258066 non-null  object        
 9   LOCATION                    52612 non-null   object        
 10  LO

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,ITEMID,VALUE,VALUEUOM,LOCATION,...,ORDERCATEGORYNAME,SECONDARYORDERCATEGORYNAME,ORDERCATEGORYDESCRIPTION,ISOPENBAG,CONTINUEINNEXTDEPT,CANCELREASON,STATUSDESCRIPTION,COMMENTS_EDITEDBY,COMMENTS_CANCELEDBY,COMMENTS_DATE
0,1552,30139,191230,250393,2130-01-29 18:51:00,2130-02-08 08:00:00,225792,13749.0,hour,None,...,Ventilation,None,Task,1,0,0,Paused,None,None,NaT
1,19043,41034,112275,204841,2128-12-20 14:50:00,2128-12-21 09:25:00,225792,1115.0,min,None,...,Ventilation,None,Task,0,0,0,Paused,None,None,NaT
2,56408,45391,103165,257349,2135-07-15 15:18:00,2135-07-15 21:15:00,225792,357.0,day,None,...,Ventilation,None,Task,0,0,0,Paused,None,None,NaT
3,67009,54348,123562,291652,2164-09-17 07:30:00,2164-09-17 08:16:00,225792,46.0,min,None,...,Ventilation,None,Task,1,0,0,Paused,None,None,NaT
4,105320,91648,125521,224948,2180-03-15 09:30:00,2180-03-25 08:01:00,225792,14311.0,day,None,...,Ventilation,None,Task,0,0,0,Paused,None,None,NaT
5,198923,86193,137949,219173,2189-09-07 12:30:00,2189-09-07 13:15:00,225792,45.0,min,None,...,Ventilation,None,Task,0,0,0,Paused,None,None,NaT
6,218157,74727,158815,290586,2188-12-28 10:00:00,2188-12-28 11:18:00,225792,78.0,min,None,...,Ventilation,None,Task,0,0,0,Paused,None,None,NaT
7,218158,74727,158815,290586,2188-12-28 11:28:00,2188-12-28 15:21:00,225792,233.0,min,None,...,Ventilation,None,Task,0,0,0,Paused,None,None,NaT
8,218159,74727,158815,290586,2188-12-28 15:42:00,2188-12-28 22:51:00,225792,429.0,min,None,...,Ventilation,None,Task,0,0,0,Paused,None,None,NaT
9,230378,61898,170625,292099,2123-04-20 01:24:00,2123-04-20 06:00:00,225792,276.0,min,None,...,Ventilation,None,Task,0,0,0,Paused,None,None,NaT


In [ ]:
# reviewing missingness across the admissions dataset
procedureevents_null_values_df = procedureevents_mv_df.isnull().sum().reset_index()
procedureevents_null_values_df.columns = ['Column', 'Missing Count']
procedureevents_null_values_df.sort_values(by='Missing Count', ascending=False)

,Column,Missing Count
16,SECONDARYORDERCATEGORYNAME,258066
22,COMMENTS_EDITEDBY,255973
23,COMMENTS_CANCELEDBY,252377
24,COMMENTS_DATE,250284
10,LOCATIONCATEGORY,205454
9,LOCATION,205454
3,ICUSTAY_ID,88
0,ROW_ID,0
1,SUBJECT_ID,0
8,VALUEUOM,0


In [ ]:
# Count the number of lab events per HADM_ID
procedureevents_mv_per_hadm = procedureevents_mv_df.groupby('HADM_ID').size()

# Calculate the average number of lab events per HADM_ID
average_procedureevents_mv = procedureevents_mv_per_hadm.mean()
print('Average number of procedure events mv per HADM_ID: ' + str(average_procedureevents_mv))

# Calculate the average number of lab events per HADM_ID
min_procedureevents_mv = procedureevents_mv_per_hadm.min()
print('Min number of procedure events mv per HADM_ID:' + str(min_procedureevents_mv))

# Calculate the average number of lab events per HADM_ID
max_procedureevents_mv = procedureevents_mv_per_hadm.max()
print('Max number of procedure events mv per HADM_ID:' + str(max_procedureevents_mv))

Average number of procedure events mv per HADM_ID: 11.787064949301179
Min number of procedure events mv per HADM_ID:1
Max number of procedure events mv per HADM_ID:242


In [ ]:
# merge against item id for interpretation
joined_inputevents_cv_d_item_df = inputevents_cv_df.merge(d_items_df[['ITEMID', 'LABEL']], on='ITEMID', how='left')

# print top inputevents for understanding
joined_inputevents_cv_d_item_df.sort_values(by='ITEMID_COUNT', ascending=False).head(100)

,ITEMID,ITEMID_COUNT,PATIENT_COUNT,LABEL
306,30013,2557507,21418,D5W
337,30018,2392372,18353,.9% Normal Saline
1143,30131,924614,10502,Propofol
519,30045,825758,6274,Insulin
379,30025,813242,5138,Heparin
1062,30118,780555,3234,Fentanyl
1120,30128,554582,6432,Neosynephrine-k
1096,30124,505509,2288,Midazolam
1075,30120,476971,3814,Levophed-k
1199,30140,373023,10084,N/A


PROCEDURES_ICD DATASET

In [ ]:
# SQL query to extract procedure events from BigQuery
procedures_icd_query = f"""
SELECT *
FROM `physionet-data.mimiciii_clinical.procedures_icd`
"""

In [ ]:
procedures_icd_df = load_file_results_to_df(query=procedures_icd_query, bucket=gcs_bucket, client=bq_client)

Query executed and loaded to DataFrame using BigQuery
DataFrame with 240095 row(s) has been returned.


In [ ]:
procedures_icd_df.info()
procedures_icd_df.head(10)
procedures_icd_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240095 entries, 0 to 240094
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   row_id      240095 non-null  Int64 
 1   subject_id  240095 non-null  Int64 
 2   hadm_id     240095 non-null  Int64 
 3   seq_num     240095 non-null  Int64 
 4   icd9_code   240095 non-null  object
dtypes: Int64(4), object(1)
memory usage: 10.1+ MB


,row_id,subject_id,hadm_id,seq_num
count,240095.0,240095.0,240095.0,240095.0
mean,120048.0,32851.481472,150082.948591,4.388434
std,69309.600778,27771.642798,28918.380367,3.772549
min,1.0,2.0,100003.0,1.0
25%,60024.5,11615.0,125186.0,2.0
50%,120048.0,23450.0,149988.0,3.0
75%,180071.5,51951.0,175340.0,6.0
max,240095.0,99999.0,199999.0,40.0


In [ ]:
# reviewing missingness across the admissions dataset
procedures_icd_null_values_df = procedures_icd_df.isnull().sum().reset_index()
procedures_icd_null_values_df.columns = ['Column', 'Missing Count']
procedures_icd_null_values_df.sort_values(by='Missing Count', ascending=False)

,Column,Missing Count
0,row_id,0
1,subject_id,0
2,hadm_id,0
3,seq_num,0
4,icd9_code,0


In [ ]:
# merging clinical codes to their written references for understanding
merged_procedures_icd_ref_df = procedures_icd_df.merge(d_icd_procedures_df[['icd9_code', 'short_title']], on='icd9_code', how='left')

# count the occurrences of each procedure by itemid and the number of distinct patients that have had that procedure
procedures_icd_count_df = merged_procedures_icd_ref_df.groupby(['icd9_code', 'short_title']).agg(
    icd9_code_count = ('icd9_code', 'count'),
    unique_patients_count = ('subject_id', 'nunique')
).reset_index()

#  sort and print
procedures_icd_count_df.sort_values(by='icd9_code_count', ascending=False)

,icd9_code,short_title,icd9_code_count,unique_patients_count
612,3893,Venous cath NEC,14731,11212
1875,9604,Insert endotracheal tube,10333,9174
1906,966,Entral infus nutrit sub,9300,8275
1908,9671,Cont inv mec ven <96 hrs,9100,8367
1965,9904,Packed cell transfusion,7244,6368
645,3961,Extracorporeal circulat,6838,6716
1909,9672,Cont inv mec ven 96+ hrs,6048,5473
1993,9955,Vaccination NEC,5842,5810
1772,8856,Coronar arteriogr-2 cath,5337,4929
610,3891,Arterial catheterization,4737,4422
